In [8]:
import requests
import pandas as pd
import numpy as np
import json
from config import api_key

In [ ]:
standing_data = requests.get(team_url).json()
def extract_team_data(data):
    teams = []
    if 'teams' in data:
        for team in data['teams']:
            team_id = team['id']
            team_name = team['name']
            team_wins = team['wins']
            team_losses = team['losses']
            teams.append({'id': team_id, 'name': team_name, team_wins: 'wins', team_losses: 'losses'})
    if 'divisions' in data:
        for division in data['divisions']:
            teams += extract_team_data(division)
    if 'conferences' in data:
        for conference in data['conferences']:
            teams += extract_team_data(conference)
    return teams

# Extract team id and team name from the team_data dictionary
teams = extract_team_data(standing_data)

# Create DataFrame from the extracted team data
team_info_df = pd.DataFrame(teams)

# Display the DataFrame
print(team_info_df)

#create list of IDs for looping
team_id_list = []
team_name_list = []
for i in range(len(team_info_df)):
    team_id_list.append(team_info_df.loc[i, 'id'])
    team_name_list.append(team_info_df.loc[i, 'name'])

In [ ]:
results_dict = {}
#loop through all teams
for i in range(len(team_id_list)):
    team_id_num = team_id_list[i]
    target_url = f'http://api.sportradar.us/nba/trial/v8/en/seasons/2022/REG/teams/{team_id_num}/statistics.json?api_key={api_key}'
    indiv_team_data = requests.get(target_url).json()

    #get team stats
    team_totals_dict = indiv_team_data['own_record']['total']
    opp_vs_team_totals_dict = indiv_team_data['opponents']['total']

    team_stat_keys = ['games_played', 'minutes', 'field_goals_made', 'field_goals_att',
                      'three_points_made', 'three_points_att', 'three_points_pct',
                      'free_throws_made', 'free_throws_att', 'offensive_rebounds',
                      'defensive_rebounds', 'rebounds', 'assists', 'turnovers',
                      'steals', 'blocks', 'points']
    team_stats = {k: v for k, v in team_totals_dict.items() if k in team_stat_keys}
    opp_vs_team_stats = {k: v for k, v in opp_vs_team_totals_dict.items() if k in team_stat_keys}

    #team stats calculations
    poss = 0.96 * ((team_stats['field_goals_att']) - (team_stats['offensive_rebounds']) + (team_stats['turnovers']) + 0.44 * (team_stats['free_throws_att']))
    team_o_rtg = 100 * (team_stats['points']) / (poss)
    team_d_rtg = 100 * (opp_vs_team_stats['points']) / (poss)
    team_net_rtg = (team_o_rtg) - (team_d_rtg)

    #store results
    team_name = team_name_list[i]
    results_dict[team_name] = {
        'poss': poss,
        'team_o_rtg': team_o_rtg,
        'team_d_rtg': team_d_rtg,
        'team_net_rtg': team_net_rtg 
    }

    time.sleep(1)

    

results_df = pd.DataFrame.from_dict(results_dict).T


results_df